In [0]:
# --- 1. LIMPIEZA DE TABLAS DELTA ---
tablas = [
    "bronze_nyc_taxi_2025_enero", 
    "bronze_taxi_zones", 
    "silver_nyc_taxi", 
    "gold_taxi_kpis",
    "nyc_taxi_2025_enero"
]

print("🧹 Eliminando tablas del catálogo...")
for tabla in tablas:
    spark.sql(f"DROP TABLE IF EXISTS {tabla}")

# --- 2. LIMPIEZA DE ARCHIVOS TEMPORALES EN DISCO ---
import os
import shutil

temp_path = "/tmp/fhvhv_tripdata_2025-01.parquet"
if os.path.exists(temp_path):
    print(f"🗑️ Eliminando archivo temporal: {temp_path}")
    os.remove(temp_path)

# --- 3. REINICIO DE WIDGETS ---
print("🔄 Reiniciando parámetros de interfaz...")
dbutils.widgets.removeAll()

print("\n✨ Entorno limpio. Por favor, vuelve a ejecutar el notebook '00_config' e ingresa tus llaves.")

In [0]:
%run ./00_config

In [0]:
# 1. Borrar tablas para empezar limpio
tablas = ["bronze_nyc_taxi_2025_enero", "bronze_taxi_zones", "silver_nyc_taxi", "gold_taxi_kpis"]
for t in tablas:
    spark.sql(f"DROP TABLE IF EXISTS {t}")

# 2. Borrar archivo físico temporal que puede estar corrupto
import os
local_path = "/tmp/fhvhv_tripdata_2025-01.parquet"
if os.path.exists(local_path):
    os.remove(local_path)
    print(f"✅ Archivo {local_path} eliminado.")

# 3. Limpiar widgets
dbutils.widgets.removeAll()
print("✨ Entorno reseteado. Vuelve a ejecutar el config.")

In [0]:
%sql
-- Celda de tipo SQL
SHOW TABLES;

In [0]:
# --- Validación de IDs eliminados en el proceso Silver ---
bronze_table = "workspace.default.bronze_nyc_taxi_2025_enero"
silver_table = "workspace.default.silver_nyc_taxi"

df_bronze = spark.table(bronze_table)
df_silver = spark.table(silver_table)

# Identificamos los IDs que estaban en Bronze pero no en Silver
df_ids_eliminados = df_bronze.join(
    df_silver.select("id"), 
    df_bronze.id == df_silver.id, 
    "left_anti"
).select(df_bronze.id.alias("id_eliminado"))
display(df_ids_eliminados)


In [0]:
%sql
-- Validacion de por que se eliminaron los id's, con el fin de garantizar el correcto proceso de limpieza de los datos --
-- Estos registros se eliminan por que en las columnas pickup_datetime y dropoff_datetime tienen la misma fecha y hora y en el analisis se calcula la distancia de recorrido de los servicios --

select * from bronze_nyc_taxi_2025_enero where id in ('95a57e69-9b81-4a1f-86c9-55838468c369',
'f532a55f-9a65-4ad1-b870-2e0d97782e73',
'fde4bd57-ba81-42a1-b6f1-39a875b08650',
'219f39de-6bdb-4a58-8c5e-0373afd17216'
)

In [0]:
%sql
-- Esta es una validacion de unos registros eliminados por duplicidad donde se evidencio que se debia agregar otra columna para complementar la llave que realiza la duplicidad, la columna es reques_datetime confirmando que la data se encuentra muy pulida
select * from bronze_nyc_taxi_2025_enero where hvfhs_license_num = 'HV0003'
and pickup_datetime = '2025-01-03T18:45:58.000+00:00'
and PULocationID = 130
and DOLocationID = 134
and driver_pay = 6.72